<a href="https://colab.research.google.com/github/Sapryka48/Saprykin-s-Algorithmization/blob/main/_crypto_price_interactive_dashboard_daily_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ВАЖНО: ЗАПУСТИТЕ ЭТУ ЯЧЕЙКУ, ЧТОБЫ ИМПОРТИРОВАТЬ ВАШИ ИСТОЧНИКИ ДАННЫХ KAGGLE,
# ЗАTEM ВЫ МОЖЕТЕ УДАЛИТЬ ЭТУ ЯЧЕЙКУ.
# ПРИМЕЧАНИЕ: ЭТА СРЕДА ЗАПИСИ ОТЛИЧАЕТСЯ ОТ PYTHON-СРЕДЫ KAGGLE,
# ПОЭТОМУ ВАШЕЙ ЯЧЕЙКЕ МОГУТ НЕ ХВАТАТЬ БИБЛИОТЕК
import kagglehub # Импортируем библиотеку kagglehub, которая позволяет загружать наборы данных из Kaggle.
# Загружаем набор данных "all-crypto-currencies" от пользователя jessevent
jessevent_all_crypto_currencies_path = kagglehub.dataset_download('jessevent/all-crypto-currencies')
# Загружаем набор данных "cryptocurrencypricehistory" от пользователя sudalairajkumar
sudalairajkumar_cryptocurrencypricehistory_path = kagglehub.dataset_download('sudalairajkumar/cryptocurrencypricehistory')
# Загружаем набор данных "top-50-cryptocurrency-historical-prices" от пользователя odins0n
odins0n_top_50_cryptocurrency_historical_prices_path = kagglehub.dataset_download('odins0n/top-50-cryptocurrency-historical-prices')

print('Data source import complete.') # Выводим сообщение о завершении импорта источников данных


100%|██████████| 22.5M/22.5M [00:01<00:00, 20.5MB/s]

Extracting files...


100%|██████████| 1.70M/1.70M [00:00<00:00, 2.94MB/s]

Extracting files...


100%|██████████| 2.06M/2.06M [00:00<00:00, 3.43MB/s]

Extracting files...
Data source import complete.


### <center> 🪙Crypto Price Interactive Dashboard(Daily Update) </center>


In [ ]:
START_DATE = "2024-01-11"  # Начальная дата для исторических данных
RSI_TIME_WINDOW = 7  # Период времени в днях для расчета RSI

# Импорт необходимых библиотек
import requests  # Библиотека для выполнения HTTP-запросов
import pandas as pd  # Библиотека для работы с данными в формате таблиц
import warnings  # Библиотека для управления предупреждениями
import datetime as dt  # Библиотека для работы с датами и временем
import plotly.express as px  # Библиотека для визуализации данных
import matplotlib.pyplot as plt  # Библиотека для создания статичных графиков
import plotly.graph_objects as go  # Библиотека для создания интерактивных графиков
import pandas_datareader.data as web  # Библиотека для получения данных из различных источников
from plotly.subplots import make_subplots  # Функция для создания подграфиков
warnings.filterwarnings('ignore')  # Игнорируем предупреждения

# Список URL-адресов для скачивания исторических данных о криптовалютах
urls = [
    "https://www.cryptodatadownload.com/cdd/Bitfinex_EOSUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_EDOUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_BTCUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_ETHUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_LTCUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_BATUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_OMGUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_DAIUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_ETCUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_ETPUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_NEOUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_REPUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_TRXUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_XLMUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_XMRUSD_d.csv",
    "https://www.cryptodatadownload.com/cdd/Bitfinex_XVGUSD_d.csv",
]

# Список имен криптовалют, соответствующих URL
crypto_names = [
    "EOS Coin (EOS)",
    "Eidoo (EDO)",
    "Bitcoin (BTC)",
    "Ethereum (ETH)",
    "Litecoin (LTC)",
    "Basic Attention Token (BAT)",
    "OmiseGO (OMG)",
    "Dai (DAI)",
    "Ethereum Classic (ETC)",
    "Metaverse (ETP)",
    "Neo (NEO)",
    "Augur (REP)",
    "TRON (TRX)",
    "Stellar (XLM)",
    "Monero (XMR)",
    "Verge (XVG)"
]

## Загрузка и обработка данных
def df_loader(urls, start_date="2021-01-01"):
    filenames = []  # Список для хранения имен файлов
    all_df = pd.DataFrame()  # Пустой DataFrame для объединенных данных
    for idx, url in enumerate(urls):  # Перебираем URL-адреса
        req = requests.get(url, verify=False)  # Выполняем HTTP GET-запрос
        url_content = req.content  # Получаем содержимое ответа
        filename = url[48:]  # Извлекаем имя файла из URL
        csv_file = open(filename, 'wb')  # Создаем файл для записи
        csv_file.write(url_content)  # Записываем содержимое в файл
        csv_file.close()  # Закрываем файл
        filename = filename[:-9]  # Убираем последние 9 символов из имени файла
        filenames.append(filename)  # Добавляем имя файла в список

    for file in filenames:  # Обрабатываем все загруженные файлы
        df = pd.read_csv(file + "USD_d.csv", header=1, parse_dates=["date"])  # Загружаем CSV-файл
        df = df[df["date"] > start_date]  # Фильтруем строки по дате
        df.index = df.date  # Устанавливаем колонку "date" как индекс
        df.drop(labels=[df.columns[0], df.columns[1], df.columns[8]], axis=1, inplace=True)  # Удаляем ненужные колонки
        all_df = pd.concat([all_df, df], ignore_index=False)  # Объединяем данные всех файлов

    return all_df, filenames  # Возвращаем объединенный DataFrame и список имен файлов

def computeRSI(data, time_window):  # Функция для расчета индикатора RSI
    diff = data.diff(1).dropna()  # Вычисляем разницу между текущими и предыдущими ценами
    up_chg = 0 * diff  # Инициализируем массив для увеличений
    down_chg = 0 * diff  # Инициализируем массив для уменьшений
    up_chg[diff > 0] = diff[diff > 0]  # Заполняем увеличения
    down_chg[diff < 0] = diff[diff < 0]  # Заполняем уменьшения
    up_chg_avg = up_chg.ewm(com=time_window-1, min_periods=time_window).mean()  # Расчет экспоненциальной скользящей средней для увеличений
    down_chg_avg = down_chg.ewm(com=time_window-1, min_periods=time_window).mean()  # Расчет экспоненциальной скользящей средней для уменьшений
    rs = abs(up_chg_avg / down_chg_avg)  # Расчет относительной силы
    rsi = 100 - 100 / (1 + rs)  # Формула для расчета RSI
    return rsi  # Возвращаем рассчитанный RSI

# Загрузка данных с использованием функции df_loader
all_df, filenames = df_loader(urls, start_date=START_DATE)

crypto_df = []  # Список для хранения DataFrame каждой криптовалюты
for file in filenames:  # Перебираем имена файлов
    symbol = file + str("/USD")  # Формируем символ криптовалюты
    temp_df = pd.DataFrame(all_df[all_df["symbol"] == symbol])  # Фильтруем DataFrame по символу криптовалюты
    temp_df.drop(columns=["symbol"], inplace=True)  # Удаляем колонку "symbol"
    temp_df["close_rsi"] = computeRSI(temp_df['close'], time_window=RSI_TIME_WINDOW)  # Добавляем колонку RSI для закрытия
    temp_df["high_rsi"] = 30  # Устанавливаем уровень 30 для RSI
    temp_df["low_rsi"] = 70  # Устанавливаем уровень 70 для RSI
    exec('%s = temp_df.copy()' % file.lower())  # Создаем копию DataFrame и присваиваем имя переменной по символу
    crypto_df.append(temp_df)  # Добавляем временный DataFrame в список

## Создание графиков
fig = make_subplots(rows=3,  # Задаем количество строк для подграфиков
                    cols=2,  # Задаем количество столбцов для подграфиков
                    shared_xaxes=True,  # Разрешаем общую ось x для всех подграфиков
                    specs=[[{"rowspan": 2}, {"rowspan": 2}],  # Настраиваем сетку подграфиков
                           [{"rowspan": 1}, {"rowspan": 1}],
                           [{}, {}]
                          ])

# Кнопки для выбора диапазона дат на графиках
date_buttons = [
    {'step': "all", 'label': "All time"},  # Отображение всех данных
    {'count':  1, 'step': "year", 'stepmode': "backward", 'label': "Last Year"},  # Предыдущий год
    {'count':  1, 'step': "year", 'stepmode': "todate", 'label': "Current Year"},  # Текущий год
    {'count':  1, 'step': "month", 'stepmode': "backward", 'label': "Last 2 Months"},  # Последние 2 месяца
    {'count':  1, 'step': "month", 'stepmode': "todate", 'label': "Current Month"},  # Текущий месяц
    {'count':  7, 'step': "day", 'stepmode': "todate", 'label': "Current Week"},  # Текущая неделя
    {'count':  4, 'step': "day", 'stepmode': "backward", 'label': "Last 4 days"},  # Последние 4 дня
    {'count':  1, 'step': "day", 'stepmode': "backward", 'label': "Today"},  # Сегодня
]

buttons = []  # Список для кнопок выбора активных графиков
i = 0  # Индекс для криптовалют
j = 0  # Индекс для скрытых графиков
COUNT = 8  # Количество графиков на одну криптовалюту
vis = [False] * len(crypto_names) * COUNT  # Инициализация видимости графиков

for df in crypto_df:  # Перебираем DataFrame каждой криптовалюты
    for k in range(COUNT):  # Устанавливаем видимость графиков
        vis[j + k] = True
    buttons.append({
        'label': crypto_names[i],  # Название кнопки
        'method': 'update',  # Метод изменения видимости
        'args': [{'visible': vis},  # Обновляем видимость графиков
                  {'title': crypto_names[i] + ' Charts and Indicators'}]  # Обновляем заголовок графика
    })
    i += 1  # Увеличиваем индекс криптовалюты
    j += COUNT  # Увеличиваем индекс для видимости
    vis = [False] * len(crypto_names) * COUNT  # Обнуляем видимость

for df in crypto_df:  # Добавляем графики для каждой криптовалюты
    fig.add_trace(
        go.Candlestick(x=df.index,  # Ось x – индекс DataFrame (даты)
                       open=df['open'],  # Цены открытия
                       high=df['high'],  # Высокие цены
                       low=df['low'],    # Низкие цены
                       close=df['close'],  # Цены закрытия
                       showlegend=False  # Не показываем легенду
                       ),
        row=1,  # Указываем номер строки
        col=1   # Указываем номер столбца
    )
    fig.add_trace(
        go.Bar(x=df.index,  # Ось x – индекс DataFrame (даты)
               y=df["Volume USD"],  # Объем в долларах США
               showlegend=False,  # Не показываем легенду
               marker_color='aqua'),  # Цвет столбцов
        row=3,  # Указываем номер строки
        col=1   # Указываем номер столбца
    )

    # Добавление графиков к фигуре (subplot)

fig.add_trace(  # Добавляем линию для закрытия цены (close)
    go.Scatter(x=df.index,  # Ось x - индекс DataFrame (даты)
               y=df['close'],  # Ось y - цены закрытия
               mode='lines',  # Рисуем линию
               showlegend=False,  # Не показывать легенду
               line=dict(color="red", width=4)),  # Цвет линии - красный, ширина - 4
    row=1,  # Расположение в первой строке
    col=2   # И во втором столбце
)

# Добавляем линию для минимальных цен (low) с затенением
fig.add_trace(
    go.Scatter(x=df.index,  # Ось x - индекс DataFrame (даты)
               y=df['low'],  # Ось y - минимальные цены
               fill='tonexty',  # Затенение области до следующей линии (нужно для заполнения между линиями)
               mode='lines',  # Рисуем линию
               showlegend=False,  # Не показывать легенду
               line=dict(width=2, color='pink', dash='dash')),  # Цвет линии - розовый, пунктирная линия
    row=1,  # Расположение в первой строке
    col=2   # И во втором столбце
)

# Добавляем линию для максимальных цен (high) с затенением
fig.add_trace(
    go.Scatter(x=df.index,  # Ось x - индекс DataFrame (даты)
               y=df['high'],  # Ось y - максимальные цены
               fill='tonexty',  # Затенение области до следующей линии
               mode='lines',  # Рисуем линию
               showlegend=False,  # Не показывать легенду
               line=dict(width=2, color='pink', dash='dash')),  # Цвет линии - розовый, пунктирная линия
    row=1,  # Расположение в первой строке
    col=2   # И во втором столбце
)

# Добавляем линию для RSI
fig.add_trace(
    go.Scatter(x=df.index,  # Ось x - индекс DataFrame (даты)
               y=df['close_rsi'],  # Ось y - значения RSI для закрытия
               mode='lines',  # Рисуем линию
               showlegend=False,  # Не показывать легенду
               line=dict(color="aquamarine", width=4)),  # Цвет линии - аквамариновый, ширина - 4
    row=3,  # Расположение в третьей строке
    col=2   # И во втором столбце
)

# Добавляем линии для низких и высоких уровней RSI с затенением
fig.add_trace(
    go.Scatter(x=df.index,  # Ось x - индекс DataFrame (даты)
               y=df['low_rsi'],  # Ось y - низкий уровень RSI
               fill='tonexty',  # Затенение области до следующей линии
               mode='lines',  # Рисуем линию
               showlegend=False,  # Не показывать легенду
               line=dict(width=2, color='aqua', dash='dash')),  # Цвет линии - аквамариновый, пунктирная линия
    row=3,  # Расположение в третьей строке
    col=2   # И во втором столбце
)

fig.add_trace(
    go.Scatter(x=df.index,  # Ось x - индекс DataFrame (даты)
               y=df['high_rsi'],  # Ось y - высокий уровень RSI
               fill='tonexty',  # Затенение области до следующей линии
               mode='lines',  # Рисуем линию
               showlegend=False,  # Не показывать легенду
               line=dict(width=2, color='aqua', dash='dash')),  # Цвет линии - аквамариновый, пунктирная линия
    row=3,  # Расположение в третьей строке
    col=2   # И во втором столбце
)


fig.update_xaxes(
    #Задает шрифт для меток (подписей) на оси X. Шрифт имеет размер 15, семейство 'monospace' и цвет '#B8B8B8'.
    tickfont=dict(size=15, family='monospace', color='#B8B8B8'),
    #
    tickmode='array',
    #Указывает режим работы меток. Установка на 'array' позволяет определить фиксированный массив меток.
    ticklen=6,
    #: Задает длину меток на оси X. В данном случае — 6.
    showline=False,
    #Указывает, нужно ли отображать линию оси. Здесь она отключена (False).
    showgrid=True,
    #Включает отображение сетки. True означает, что сетка будет видна на графике
    gridcolor='#595959',
    #Задает цвет сетки. В данном примере используется цвет '#595959'.
    ticks='outside'  )
    # Устанавливает внешний вид меток. Значение 'outside' означает, что метки будут находиться за пределами графика.

# Обновляем настройки оси X
fig.update_xaxes(
    tickfont=dict(size=15, family='monospace', color='#B8B8B8'),  # Настройки шрифта для меток по оси X
    tickmode='array',  # Режим отображения меток
    ticklen=6,  # Длина меток по оси X
    showline=False,  # Не показывать линию оси
    showgrid=True,  # Показывать сетку на графике
    gridcolor='#595959',  # Цвет сетки
    ticks='outside'  # Расположение меток снаружи
)

# Обновляем настройки графика
fig.update_layout(
    spikedistance=100,  # Дистанция до спайков (индикаторов на графике)
    xaxis_rangeslider_visible=False,  # Скрыть ползунок диапазона по оси X
    hoverdistance=1000  # Дистанция, на которой появляется всплывающее окно
)

# Устанавливаем, чтобы показ спайков был включен на оси X
fig.update_xaxes(
    showspikes=True,  # Включить отображение спайков
    spikesnap="cursor",  # Привязка спайка к курсору
    spikemode="across"  # Режим спайка через всю графику
)

# Обновляем настройки оси Y
fig.update_yaxes(
    showspikes=True,  # Включить отображение спайков на оси Y
    spikesnap='cursor',  # Привязка спайка к курсору
    spikemode="across"  # Режим спайков через всю графику
)

# Обновляем настройки оси Y для визуализации
fig.update_yaxes(
    tickfont=dict(size=15, family='monospace', color='#B8B8B8'),  # Настройки шрифта для меток по оси Y
    tickmode='array',  # Режим отображения меток
    showline=False,  # Не показывать линию оси
    ticksuffix='$',  # Добавить символ '$' в конце меток
    showgrid=True,  # Показывать сетку на графике
    gridcolor='#595959',  # Цвет сетки
    ticks='outside'  # Расположение меток снаружи
)

# Обновляем общий макет графика
fig.update_layout(
    width=1120,  # Ширина графика
    height=650,  # Высота графика
    font_family='monospace',  # Шрифт для текста на графике
    xaxis=dict(rangeselector=dict(buttons=date_buttons)),  # Кнопки выбора диапазона дат на оси X
    updatemenus=[dict(type='dropdown',  # Добавляем выпадающее меню
                      x=1.0,  # Положение по оси X
                      y=1.108,  # Положение по оси Y
                      showactive=True,  # Подсветка выбранного элемента
                      active=2,  # Активный элемент
                      buttons=buttons)],  # Кнопки для выбора криптовалюты
    title=dict(text='<b>Cryptocurrencies Dashboard</b>',  # Заголовок графика
               font=dict(color='#FFFFFF', size=22),  # Цвет и размер шрифта заголовка
               x=0.50),  # Центрирование заголовка по оси X
     font=dict(color="blue"),  # Цвет шрифта на графике
    annotations=[  # Аннотации на графике
        dict(text="<b>Choose Cryptocurrency: <b>",  # Текст аннотации
             font=dict(size=20, color="#ffffff"),  # Настройки шрифта аннотации
             showarrow=False,  # Не показывать стрелку
             x=1.02,  # Положение по оси X
             y=1.20,  # Положение по оси Y
             xref='paper', yref="paper",  # Ссылка на координаты в пространстве графика
             align="left"),  # Выравнивание текста
        dict(text="<b>Candlestick Chart <b>",  # Текст аннотации
             font=dict(size=20, color="#ffffff"),  # Настройки шрифта
             showarrow=False,  # Не показывать стрелку
             x=0.12,  # Положение по оси X
             y=0.285,  # Положение по оси Y
             xref='paper', yref="paper",  # Ссылка на координаты
             align="left"),
        dict(text="<b>Price Chart<b>",  # Текст аннотации
             font=dict(size=20, color="#ffffff"),
             showarrow=False,
             x=0.82,
             y=0.285,
             xref='paper', yref="paper",
             align="left"),
        dict(text="<b>Volume Traded<b>",  # Текст аннотации
             font=dict(size=20, color="#ffffff"),
             showarrow=False,
             x=0.14,
             y=-0.53,
             xref='paper', yref="paper",
             align="left"),
        dict(text="<b>Relative Strength Index (RSI)<b>",  # Текст аннотации
             font=dict(size=20, color="#ffffff"),
             showarrow=False,
             x=0.94,
             y=-0.53,
             xref='paper', yref="paper",
             align="left")
    ],
    template="plotly_dark"  # Шаблон для графика
)
# Скрываем все данные на графике для всех метрик по умолчанию
for i in range(0, 16 * COUNT):  # Перебираем все графики (16 для каждой криптовалюты)
    fig.data[i].visible = False  # Устанавливаем видимость данных на False

# Показываем данные только для первых COUNT графиков
for i in range(COUNT):  # Перебираем количество графиков, которые хотим отобразить
    fig.data[i].visible = True  # Устанавливаем видимость для этих данных на True

# Настройки отображения ползунка диапазона по оси X
fig.layout["xaxis"]["rangeslider"]["visible"] = False  # Скрыть ползунок диапазона для главной оси X
fig.layout["xaxis2"]["rangeslider"]["visible"] = False  # Скрыть ползунок диапазона для оси X2 (если такая имеется)
fig.layout["xaxis5"]["rangeslider"]["visible"] = True  # Показать ползунок диапазона для оси X5
fig.layout["xaxis6"]["rangeslider"]["visible"] = True  # Показать ползунок диапазона для оси X6

# Настройка внешнего вида ползунков диапазона
fig.layout["xaxis5"]["rangeslider"]["borderwidth"] = 4  # Установить ширину границы для ползунка X5
fig.layout["xaxis6"]["rangeslider"]["borderwidth"] = 4  # Установить ширину границы для ползунка X6
fig.layout["xaxis5"]["rangeslider"]["bordercolor"] = "aqua"  # Установить цвет границы для ползунка X5
fig.layout["xaxis6"]["rangeslider"]["bordercolor"] = "aqua"  # Установить цвет границы для ползунка X6

# Настройки для оси Y6
fig.layout["yaxis6"]["ticksuffix"] = ""  # Установить пустой суффикс для меток на оси Y6
fig.layout["yaxis6"]["range"] = [10, 100]  # Установить диапазон значений для оси Y6 от 10 до 100

# Отобразить график
fig.show()

* Click the 🏠 icon on the RIGHT TOP to reset the dashboard.
* Start date for historical data is 1st January 2024.
* Total 16 cryptocurrencies are used in the dashboard. Check them [here](https://www.cryptodatadownload.com/data/bitfinex/).
* Relative Strength Index (RSI) has a period of 7 days. Find more about RSI [here](https://en.wikipedia.org/wiki/Relative_strength_index).

### <center>If you find this notebook useful, support with an upvote👍</center>